# Java ODBCデータソースを登録

* ODBC(Open Database Connectivity)
    * リレーショナルデータベース管理システム (RDBMS) にアクセスするための共通インタフェース (API)
    * マイクロソフトによって提唱された
* JDBC(Java Database Connectivity)
    * Javaとリレーショナルデータベースの接続のためのAPI
* Javaで利用できるJDBCドライバ
    * Type1 JDBC-ODBC ブリッジドライバ
        * JDBCからの要求をODBCを経由してデータベースへアクセスする
        * 先にODBCデータソースを登録する必要がある
        * <font color="red">Java8では標準から削除された</font>
            * 今日では多くのデータベースがJDBCドライバを出しており、ODBCを経由させる必要性はほとんどなくなったため
    * Type2 Native ブリッジドライバ
    * Type3 ネットプロトコルドライバ
    * Type4 ネイティブプロトコルドライバ
        * PostgreSQL JDBC driverはこれ

## PostgreSQLのドライバのインストール

[PostgreSQL JDBC driver](https://jdbc.postgresql.org/download.html) よりjarファイルをダウンロード

In [2]:
%%bash
### jarファイルをダウンロード
#wget https://jdbc.postgresql.org/download/postgresql-42.1.1.jar

--2017-05-31 03:01:10--  https://jdbc.postgresql.org/download/postgresql-42.1.1.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 174.143.35.228, 2001:4800:1501:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|174.143.35.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712452 (696K) [application/java-archive]
Saving to: 'postgresql-42.1.1.jar'

     0K .......... .......... .......... .......... ..........  7%  157K 4s
    50K .......... .......... .......... .......... .......... 14%  312K 3s
   100K .......... .......... .......... .......... .......... 21% 11.6M 2s
   150K .......... .......... .......... .......... .......... 28%  330K 2s
   200K .......... .......... .......... .......... .......... 35% 5.91M 1s
   250K .......... .......... .......... .......... .......... 43%  114M 1s
   300K .......... .......... .......... .......... .......... 50% 11.6M 1s
   350K .......... .......... .......... .......... .......... 57% 11

## DBに接続

In [1]:
%%bash
### 変数の設定
name=PostgresConnect
classpath=".:/root/git_jupyter_notebook/Java/postgresql-42.1.1.jar"

### ソースの編集
cat <<- EOS > ${name}.java
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.ResultSetMetaData;
import java.sql.Statement;
import java.util.ArrayList;
import java.util.List;

public class ${name} {
  public static void main(String[] args) throws Exception{
    Connection connection = null;
    Statement statement = null;
    ResultSet resultSet = null;
    
    try{
      //# 接続。pg_shadowを表示するため管理者(postgres)
      connection = DriverManager.getConnection("jdbc:postgresql://10.136.3.189:5432/jsd_test", "postgres", "pass");
      statement = connection.createStatement();
      
      //# SQL文発行
      resultSet = statement.executeQuery("SELECT * FROM pg_shadow");

      //# 値の取得。resultSetから値を取り出している。fieldsに取り出した値を1行ずつ追加している
      List<String> fields = new ArrayList<String>();
      ResultSetMetaData rsmd = resultSet.getMetaData();
      for (int i = 1; i <= rsmd.getColumnCount(); i++) {
        fields.add(rsmd.getColumnName(i));
      }

      //# 結果の出力
      int rowCount = 0;
      while (resultSet.next()) {
        rowCount++;

        System.out.println("---------------------------------------------------");
        System.out.println("--- Rows:" + rowCount);
        System.out.println("---------------------------------------------------");

        //# 値は、「resultSet.getString(<フィールド名>)」で取得する。
        for (String field : fields) {
          System.out.println(field + ":" + resultSet.getString(field));
        }
      }
            
    } finally {
      //# 接続を切断する
      if (resultSet != null) {
        resultSet.close();
      }
      if (statement != null) {
        statement.close();
      }
      if (connection != null) {
        connection.close();
      }
    }
  }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

### 実行
java -classpath ${classpath} ${name}

---------------------------------------------------
--- Rows:1
---------------------------------------------------
usename:postgres
usesysid:10
usecreatedb:t
usesuper:t
usecatupd:t
userepl:t
passwd:md5d2743fc4ae70b03845da881b3d77b963
valuntil:null
useconfig:null
---------------------------------------------------
--- Rows:2
---------------------------------------------------
usename:prefuser
usesysid:16384
usecreatedb:f
usesuper:f
usecatupd:f
userepl:f
passwd:md548308ce61c5076c4d0d2c01676aeb78e
valuntil:null
useconfig:null
---------------------------------------------------
--- Rows:3
---------------------------------------------------
usename:jsd
usesysid:16388
usecreatedb:f
usesuper:f
usecatupd:f
userepl:f
passwd:md5075cef071f4d294944089ee00870501d
valuntil:null
useconfig:null


### メモ

* createStatement()
    * SQL文をデータベースに送るためのStatementオブジェクトを生成
    * Statementオブジェクト
        * 静的SQL文を実行し、作成された結果を返すために使用されるオブジェクト
            * デフォルトでは、Statementオブジェクトごとに、同時に開くことができるResultSetオブジェクトは1つだけ
* executeQuery(String sql)
    * 指定されたSQL文を実行する
    * 単一のResultSetオブジェクト(表のようなもの)を返す
    * ResultSet
        * データベースの結果セットを表すデータの表
        * 通常データベースに照会する文を実行することによって生成される
        * ResultSetオブジェクト
            * カーソルがデータの現在の行を指し示すよう維持
            * 初期状態では、カーソルは最初の行の先頭に配置